# 勾配消失問題
多層パーセプトロンでは、層の長さを長くすればするほど表現力は増します。一方で、学習が難しくなるという問題が知られています。

中間層が10層という深い(deepな)多層パーセプトロンを用いて問題を確認してみます。

In [ ]:
# このセルはヘルパー関数です。

# ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


# i 番目のレイヤーの出力を取得する
def getActivation(model, i, x):
    activations = K.function([model.input, K.learning_phase()], [model.layers[i].output, ])
    return activations(x)[0]


# iLayer 番目のレイヤーの勾配(誤差)を取得する
def getGradientNode(model, i, x, t):
    layer_output = model.layers[i].output
    loss = K.mean(K.binary_crossentropy(model.output, K.variable(t)))
    grads = K.gradients(loss, layer_output)[0]
    gradient_function = K.function([model.input], [grads])
    return gradient_function([x])[0]


# iLayer 番目のレイヤーにつながる重み(wij)の勾配を取得する
def getGradientParameter(model, i, x, t):
    weights = model.layers[i].weights[0]  # get only kernel (not bias)
    gradients = K.gradients(model.total_loss, weights)
    input_tensors = model._feed_inputs + model._feed_targets + model._feed_sample_weights
    gradient_function = K.function(input_tensors, gradients)
    return gradient_function([x, np.ones(len(x)), t, 0])[0]


def getRelGradientParameter(model, i, x, t):
    return getGradientParameter(model, i, x, t) / model.get_weights()[i * 2]


# モデルの勾配や活性化関数の出力等をプロット
# 左上: ウェイト(wij)の初期値
# 中上: ウェイト(wij)の微分(dE/dwij)
# 右上: ウェイト(wij)の微分(dE/dwij)の分散
# 左下: 各ノードの出力(sigma(ai))
# 中下: 各ノードでの微分(delta_ij)
# 右下: 各ノードでの微分(delta_ij)の分散
def showGradient(model, x, t):
    fig, ax = plt.subplots(2, 3, figsize=(18, 8))

    iLayers = [0, 3, 6, 10]
    labels = [
        ' 0th layer',
        ' 3th layer',
        ' 6th layer',
        'Last layer',
    ]

    values = [model.get_weights()[i * 2].flatten() for i in iLayers]
    ax[0][0].hist(values, bins=50, stacked=False, density=True, label=labels, histtype='step')
    ax[0][0].set_xlabel('weight')
    ax[0][0].set_ylabel('Probability density')
    ax[0][0].legend(loc='upper left', fontsize='x-small')

    values = [getActivation(model, i, x).flatten() for i in iLayers]
    ax[1][0].hist(values, bins=50, stacked=False, density=True, label=labels, histtype='step')
    ax[1][0].set_xlabel('activation')
    ax[1][0].set_ylabel('Probability density')
    ax[1][0].legend(loc='upper center', fontsize='x-small')

    grads = [np.abs(getGradientParameter(model, i, x, t).flatten()) for i in iLayers]
    grads = [np.log10(x[x > 0]) for x in grads]
    ax[0][1].hist(grads, bins=50, stacked=False, density=True, label=labels, histtype='step')
    ax[0][1].set_xlabel('log10(|gradient of weights|)')
    ax[0][1].set_ylabel('Probability density')
    ax[0][1].legend(loc='upper left', fontsize='x-small')

    grads = [np.abs(getGradientNode(model, i, x, t).flatten()) for i in iLayers]
    grads = [np.log10(x[x > 0]) for x in grads]
    ax[1][1].hist(grads, bins=50, stacked=False, density=True, label=labels, histtype='step')
    ax[1][1].set_xlabel('log10(|gradient of nodes|)')
    ax[1][1].set_ylabel('Probability density')
    ax[1][1].legend(loc='upper left', fontsize='x-small')

    var = [np.abs(getGradientParameter(model, i, x, t).flatten()).var() for i in range(10)]
    ax[0][2].plot(var)
    ax[0][2].set_yscale('log')
    ax[0][2].set_xlabel('Layer')
    ax[0][2].set_ylabel('Variance of gradient of weights')

    var = [np.abs(getGradientNode(model, i, x, t).flatten()).var() for i in range(10)]
    ax[1][2].plot(var)
    ax[1][2].set_yscale('log')
    ax[1][2].set_xlabel('Layer')
    ax[1][2].set_ylabel('Variance of gradient of nodes')

    plt.show()


In [ ]:
from keras.models import Sequential
from keras.initializers import RandomNormal, RandomUniform
from keras.layers import Dense

# データセットの生成
nSamples = 1000
nFeatures = 50
x = np.random.randn(nSamples, nFeatures)  # 100個の入力変数を持つイベント1000個生成。それぞれの入力変数は正規分布に従う
t = np.random.randint(2, size=nSamples).reshape([nSamples, 1])  # 正解ラベルは0 or 1でランダムに生成

# モデルの定義
activation = 'sigmoid'  # 中間層の各ノードで使う活性化関数
initializer = RandomNormal(mean=0.0, stddev=1.0)  # weight(wij)の初期値。ここでは正規分布に従って初期化する
# initializer = RandomUniform(minval=-1, maxval=1)  # weight(wij)の初期値。ここでは一様分布に従って初期化する

# 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
model = Sequential()
model.add(Dense(50, activation=activation, kernel_initializer=initializer, input_dim=nFeatures))
for i in range(9):
    model.add(Dense(50, activation=activation, kernel_initializer=initializer))
model.add(Dense(1, activation='sigmoid', kernel_initializer=initializer))
model.compile(loss='binary_crossentropy', optimizer='adam')

# モデルの勾配や活性化関数の出力等をプロット
# 左上: ウェイト(wij)の初期値
# 中上: ウェイト(wij)の微分(dE/dwij)
# 右上: ウェイト(wij)の微分(dE/dwij)の分散
# 左下: 各ノードの出力(sigma(ai))
# 中下: 各ノードでの微分(delta_ij)
# 右下: 各ノードでの微分(delta_ij)の分散
showGradient(model, x, t)


左上のプロットはパラメータ($w_{ij}$)の初期値を表しています。指定したとおり、各層で正規分布に従って初期化されています。
左下のプロットは活性化関数の出力($z_i$)を表しています。パラメータ($w_{ij}$)の初期値として正規分布を指定すると、シグモイド関数の出力はそのほとんどが0か1に非常に近い値となっています。シグモイド関数の微分は$\sigma^{'}(x)=\sigma(x)\cdot(1-\sigma(x))$なので、$\sigma(x)$が0や1に近いときは微分値も非常に小さな値となります。
誤差逆伝播の式は
$$
\begin{align}
\delta_{i}^{(k)} &= \sigma^{'}(a_i^{(k)}) \left( \sum_j w_{ij}^{(k+1)} \cdot \delta_{j}^{(k+1)} \right) \\
\frac{\partial E_n}{\partial w_{ij}^{(k)}}  &= \delta_{j}^{(k)} \cdot z_{i}^{(k)}
\end{align}
$$
でした。$\sigma^{'}(a_i^{(k)})$が小さいと後方の層から前方の層に誤差が伝わる際に、値が小さくなってしまいます。
中上、中下のプロットはそれぞれ各層での$\frac{\partial E_n}{\partial w_{ij}^{(k)}}$、$\delta_{i}^{(k)}$を表しています。
前方の層(0th layer)は後方の層と比較して分布の絶対値が小さくなっています。
右上と右下のプロットは各レイヤーでの$\frac{\partial E_n}{\partial w_{ij}^{(k)}}$と$\delta_{i}^{(k)}$の分布の分散を表しています。

このように誤差が前の層にいくにつれて小さくなるため、前の層が後ろの層と比較して学習が進まなくなります。
この問題は勾配消失の問題として知られています。

勾配消失はパラメータの初期値や、活性化関数を変更することによって解決・緩和することがわかっています。
Kerasの
- [初期化のページ](https://keras.io/ja/initializers/)
- [活性化関数のページ](https://keras.io/ja/activations/)

も参考にしながら、この問題の解決を試みてみましょう。

活性化関数・パラメータの初期化方法の変更はそれぞれコード中の"activation"、"initializer"を変更することによって行えます。

例えばパラメータの初期化を(-0.01, +0.01)の一様分布に変更するときは以下のコードのようにすれば良いです。

In [ ]:
from keras.models import Sequential
from keras.initializers import RandomNormal, RandomUniform
from keras.layers import Dense

# データセットの生成
nSamples = 1000
nFeatures = 50
x = np.random.randn(nSamples, nFeatures)  # 100個の入力変数を持つイベント1000個生成。それぞれの入力変数は正規分布に従う
t = np.random.randint(2, size=nSamples).reshape([nSamples, 1])  # 正解ラベルは0 or 1でランダムに生成

# モデルの定義
activation = 'sigmoid'  # 中間層の各ノードで使う活性化関数
# initializer = RandomNormal(mean=0.0, stddev=1.0)  # weight(wij)の初期値。ここでは正規分布に従って初期化する
initializer = RandomUniform(minval=-0.01, maxval=0.01)  # weight(wij)の初期値。ここでは一様分布に従って初期化する

# 中間層が10層の多層パーセプトロン。各レイヤーのノード数は全て50。
model = Sequential()
model.add(Dense(50, activation=activation, kernel_initializer=initializer, input_dim=nFeatures))
for i in range(9):
    model.add(Dense(50, activation=activation, kernel_initializer=initializer))
model.add(Dense(1, activation='sigmoid', kernel_initializer=initializer))
model.compile(loss='binary_crossentropy', optimizer='adam')

# モデルの勾配や活性化関数の出力等をプロット
# 左上: ウェイト(wij)の初期値
# 中上: ウェイト(wij)の微分(dE/dwij)
# 右上: ウェイト(wij)の微分(dE/dwij)の分散
# 左下: 各ノードの出力(sigma(ai))
# 中下: 各ノードでの微分(delta_ij)
# 右下: 各ノードでの微分(delta_ij)の分散
showGradient(model, x, t)


この例では活性化関数の出力が0.5付近に集中しています。
どのノードも同じ出力をしているということはノード数を増やした意味があまりなくなっており、多層パーセプトロンの表現力が十分に活かしきれていないことがわかります。
また、勾配消失も先程の例と比較して大きくなっています。